In [1]:
# 1
%matplotlib inline
import numpy as np
import pandas as pd
import os,sys
import utilities
import matplotlib.pyplot as plt
from consts import sensor_lat as lat
from consts import sensor_log as log
from consts import valid_sensor_res_10 as valid_sensor_list
from numpy import array
import time
import pickle

In [2]:
df_sub = pd.read_hdf('r_data_files/df_sub.h5','df_sub')
unix_column = 'Time since the epoch [s]'
idd = 'Station ID'

In [3]:
day_second = 86400
week_second = 7*day_second
start_list = np.arange(1162393768,1178747998+day_second,day_second)
epoch_length = 1800
for start_point in start_list:
    # start_time = time.time()
    tmp_df = df_sub[(df_sub[unix_column] > start_point) & (df_sub[unix_column] < start_point + day_second)]
    tmp_df = tmp_df.dropna()
    # 此处产生的数据都是每一个day(86400)内的
    tmp_df.to_hdf('r_data_files/day.h5',key=str(start_point),mode='a')




/home/esmidth/.pyenv/versions/anaconda3-2020.02/envs/ccs_ta/lib/python3.7/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '1162393768'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/esmidth/.pyenv/versions/anaconda3-2020.02/envs/ccs_ta/lib/python3.7/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '1162480168'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/esmidth/.pyenv/versions/anaconda3-2020.02/envs/ccs_ta/lib/python3.7/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '1162566568'; it does not match the pattern ``^[a-zA

In [4]:
del df_sub

In [3]:
from multiprocessing import Pool
import multiprocessing as mp

In [4]:
def compute(start_point):
    df = pd.read_hdf('r_data_files/day.h5',str(start_point))
    epoch_length = 30
    day_second = 86400
    time_stamps = np.arange(start_point,start_point+day_second+epoch_length,epoch_length)
    dict_list = []
    for i in range(len(time_stamps)-1):
        # print(i)
        # print(time_stamps[i],time_stamps[i+1])
        sub_df = df[(df[unix_column] > time_stamps[i]) & (df[unix_column] <= time_stamps[i+1])]
        sub_df = sub_df.dropna()
        sensor_list = sub_df[idd].values
        dict_list.append(sensor_list)
        # idd_dict = {}
        # uniq = np.unique(sensor_list)
        # for i in uniq:
        #     idd_dict[i] = 0
        # for i in sensor_list:
        #     idd_dict[i] += 1
        # # np_court = all_np(sensor_list)
        # # all_np_list.append(np_court)
        # dict_list.append(idd_dict)
        # error_num += len(diff) 
    return (dict_list,start_point)


In [5]:
manager = mp.Manager()
aggreateData = manager.dict()
start = time.time()
day_second = 86400
start_list = np.arange(1162393768,1178747998+day_second,day_second)
epoch_length = 30
output_file = open('r_data_files/3_dict_list.pkl','wb')

results = []
p = Pool(16)
for start_point in start_list:
    result = p.apply_async(func=compute,args=(start_point,))
    results.append(result)

p.close()
p.join()

print('compute_time:',time.time()-start)

output_list = []
for result in results:
    output_list.append(result.get())
pickle.dump(output_list,output_file)
output_file.close()

compute_time: 81.41365265846252
